## Ludwig, 2020 Code to Caluclate Xray Luminosity for Binary Stripped Star - Neutron Star Source.  
## Calculation by Ylva Gotberg  
  
### Dependencies :  
https://pypi.org/project/tabula-py/
and latex (for plots)

In [1]:
import tabula
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib import colors
import astropy.constants as const
import astropy.units as u

from matplotlib import rc
from matplotlib import cm

fsize = 30
# Works on ubuntu, challenging on windows
#rc("font", **{"family": "serif", "serif": ["Times"], "size": fsize})
#rc("text", usetex=True)

# Seems to work on windows.
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.serif'] = ['Times']
rcParams['font.size'] = fsize

In [2]:
# Looks like ylva used v_inf = 1.5 v_esc 

def vinf(M,R):
    return np.sqrt(2 * M * const.M_sun * const.G / (R * const.R_sun)).to(u.km/u.s)

# for m,r,v in zip(df["Mstrip"].astype(float),df["Reff"].astype(float),df["v∞"].astype(float)):
#     print((v * u.km / u.s )/vinf(m,r))

In [3]:
# From Ylva's 2018 paper 


file = "https://arxiv.org/pdf/1802.03018.pdf"
table = tabula.read_pdf(file, pages="22", multiple_tables=False, silent=True)

# Clean Up the Table, Store units in case I need it later.
df = table
units = df.iloc[0]
df = df.drop([0])
df = df.drop(["Group"], axis=1)
df["Minit"] = (
    np.array([re.findall("\d+\.\d+", str) for str in df["Minit"]])
    .flatten()
    .astype(float)
)
df

,Minit,Pinit,Mstrip,"MH,tot",log10 L,log10 LH,T?,Teff,log10 geff,Reff,"XH,s","XHe,s",log10 Ṁwind,Γe,v∞,log10 Q0,log10 Q1,log10 Q2
1,2.00,4.6,0.37,0.003,0.7,-0.9,21.7,21.6,5.55,0.17,1.00,0.00,-12.0,0.000,1370,41.2,37.4,30.8
2,2.21,4.9,0.42,0.004,1.1,-0.5,25.3,25.2,5.56,0.18,1.00,0.00,-12.0,0.001,1430,42.1,38.9,31.4
3,2.44,5.3,0.47,0.005,1.3,-0.3,27.8,27.7,5.54,0.19,1.00,0.00,-12.0,0.001,1450,42.7,40.4,35.6
4,2.70,5.6,0.54,0.006,1.5,-0.1,30.4,30.2,5.51,0.21,0.96,0.04,-12.0,0.002,1480,43.5,41.0,35.2
5,2.99,6.0,0.62,0.007,1.8,0.2,32.9,32.8,5.47,0.24,0.96,0.04,-12.0,0.003,1500,44.4,43.3,38.4
6,3.30,6.4,0.7,0.008,2.0,0.5,35.4,35.2,5.44,0.26,0.96,0.04,-12.0,0.004,1520,45.1,44.3,39.2
7,3.65,6.8,0.8,0.01,2.2,0.8,38.6,38.4,5.44,0.28,0.52,0.48,-12.0,0.005,1570,45.6,44.3,38.6
8,4.04,7.3,0.92,0.012,2.4,1.0,40.7,40.5,5.39,0.32,0.51,0.49,-11.2,0.006,1570,45.9,45.0,38.2
9,4.46,7.8,1.05,0.015,2.6,1.3,43.3,43.1,5.35,0.36,0.58,0.42,-10.5,0.009,1600,46.2,45.6,39.6
10,4.93,8.3,1.2,0.019,2.8,1.6,45.6,45.4,5.3,0.4,0.50,0.50,-10.0,0.012,1600,46.5,45.9,40.2


# $ L = \eta \epsilon \frac{G M_{ns}\dot{M}_{acc}}{R_{NS}}$  
  
# $ \dot{M}_{acc} = \frac{G^2 M_{NS}^2}{v_{relative}^3 a^2 v_{wind}} |\dot{M}_{wind}|$

# $ v_{wind}(r) = v_\infty (1 - \frac{R_{star}}{r})^\beta $

# $ v_{relative} = \sqrt{v_{wind}(r)^2 + v_{orbit}^2}$

## $\eta$ : 

In [ ]:
eta_table = tabula.read_pdf(
    "http://www.tassosfragos.com/wp-content/uploads/2014/09/apj_764_1_41.pdf",
    pages="6",
    multiple_tables=False,
    silent=True,
)
eta_table

## $\epsilon$ : 

Per https://arxiv.org/pdf/1207.3113.pdf this varies between 0.01 and 0.1. Ish? Table 1 seems more complicated... I haven't carefully read this yet.

# For Period, V_orbit, assume Kepler's laws : 
## Note: For a tight orbit, assume circular geometry, a = r (distance between orbitting object and stripped star)   
## $ P^2 = \frac{4\pi^2}{G}\frac{a^3}{(M_1 + M_2)} $  
## $ P = \frac{2\pi}{v_{orbit}} $  
## $ v_{orbit} = \frac{2\pi}{P} a $

In [ ]:
# Assuming band 0.3 - 7 KeV, absorption corrected

class xray:
    def __init__(
        self,
        table,
        period,
        M_NS=1.4 * const.M_sun,
        R_NS=10 * u.km,
        M_BH=5 * const.M_sun,
        M_WD=0.6 * const.M_sun,
        R_WD=0.005 * const.R_sun,
        beta=1,
        alpha=1.5,
        hydrogen_mass_fraction=0.02,
        v_inf_coeff = None,
        dM_wind_coeff = 0.1,
        adaf_flag = False
    ):
        
        # Power Law Exponent for v_wind   
        self.beta = beta

        self.alpha = alpha
        
        # Add units to quantities
        self.period = period * u.day

        self.M_stripped = table["Mstrip"].to_numpy(dtype=float) * const.M_sun

        self.R_stripped = table["Reff"].to_numpy(dtype=float) *  const.R_sun
        
        if v_inf_coeff: 
            
            v_esc = np.sqrt(2 * self.M_stripped * const.G / (self.R_stripped)).to(u.km/u.s)
            
            self.v_inf = v_inf_coeff * v_esc
        
        else:
        
            self.v_inf = table["v∞"].to_numpy(dtype=float) * u.km / u.s
            
        self.dM_wind = dM_wind_coeff * (10 ** table["log10 Ṁwind"].to_numpy(dtype=float)) * const.M_sun / u.year
                 
        # Opacity
        self.kramers_opacity = 0.02 * (1 + hydrogen_mass_fraction) * u.cm ** 2 / u.g
        
        # Neutron Star
        self.neutron_star_luminosity = self.calculate_luminosity(M_NS, R_NS,adaf_flag)
        
        # Black Hole
        self.R_schwarzschild = 2 * const.G * M_BH / const.c ** 2

        self.R_bh_accretion = 3 * self.R_schwarzschild

        self.black_hole_luminosity = self.calculate_luminosity(M_BH, self.R_bh_accretion,adaf_flag) 

    def calculate_luminosity(self, companion_mass, companion_radius,adaf_flag):

        G = const.G

        # Calculate "a" or distance between companion and stripped star
        self.distance = ((
            self.period ** 2 * G * (self.M_stripped + companion_mass) / (4 * np.pi ** 2)
        ) ** (1 / 3)).to(u.m)

        self.q = self.M_stripped / companion_mass
        
      
        self.roche_lobe = self.distance * (0.49 * self.q ** (2 / 3)) / (0.6 * self.q ** (2 / 3) + np.log(1 + self.q ** (1/3)))   

        # Calculate Eddington
        self.L_eddington = (
            4 * np.pi * const.c * const.G * companion_mass / self.kramers_opacity
        ).to(u.erg / u.s)

        self.dM_eddington = self.L_eddington / (const.c ** 2)

        # Calculate orbital, wind, and relative velocities
        self.omega_orbit = 2 * np.pi / self.period

        self.v_orbit = self.omega_orbit * self.distance

        self.v_wind = self.v_inf * (1 - self.R_stripped / self.distance) ** self.beta

        self.v_relative = np.sqrt(self.v_wind ** 2 + self.v_orbit ** 2)

        
        self.dM_acc = (self.alpha * G ** 2 * companion_mass ** 2 * self.dM_wind) / (
            2 * self.v_relative ** 3 * self.distance ** 2 * self.v_wind
        )

        # Calculate Uncorrected Luminosity
        self.L_uncorrected = (G * companion_mass * self.dM_acc / companion_radius).to(
            u.erg / u.s
        )

        self.epsilons, self.etas = [],[]
        for luminosity, dM_accretion in zip(self.L_uncorrected,self.dM_acc) : 
            # Calculate correction Factors eta and epsilon.

            if luminosity > 0.1 * self.L_eddington:

                epsilon = 1

            elif luminosity < 0.1 * self.L_eddington:

                epsilon = 0.1
                
                if adaf_flag:
                    epsilon = self.calculate_ADAF_efficiency(
                        dM_accretion, self.dM_eddington
                    )

            if luminosity > 0.05 * self.L_eddington:
                # High Soft State

                # Black Hole 
                if companion_mass > 3 * const.M_sun:

                    eta = 0.75

                # Neutron Star
                else:

                    eta = 0.61


            # Assuming band 0.3 - 8 KeV
            elif luminosity < 0.05 * self.L_eddington:
                # Low Hard State

                # Black Hole 
                if companion_mass > 3 * const.M_sun:

                    eta = 0.36

                # Neutron Star
                else:

                    eta = 0.29
            
            self.epsilons.append(epsilon)
            self.etas.append(eta)
         
        # Double check that the radius of the stripped star isn't the same as its roche lobe.
        
        truth = []
        
        for R, RL, M in zip (self.R_stripped, self.roche_lobe, self.M_stripped):
            
            if R >= RL : 
                            
                truth.append(np.nan)
            
            else:
                
                truth.append(1.)
            
        
        
        # Calculate Final Luminosity
        self.L = self.L_uncorrected * np.array(self.epsilons) * np.array(self.etas) * truth
        

        
            
        return self.L

        # This is epsilon

    def calculate_ADAF_efficiency(self, Mdot, Mdot_edd, delta=0.01):
        """
        Written by Ylva/Katie. 
        Calculate the mass-luminosity efficiency based on the ADAF
        model of Xie & Yuan (2012), using the piecewise fitting function
        using values provided their Table 1 for their Equation 11.

        Args:
            Mdot : mass transfer rate onto a BH [msun/year]
            Mdot_edd : Eddington mass transfer rate (same units as Mdot)
            delta : (optional) fraction of ionization energy acting on electrons

        Returns:
            epsilon : mass-light conversion efficiency
        """

        M_ratio = Mdot.to(u.erg * u.s / u.m**2) / Mdot_edd.to(u.erg * u.s / u.m**2)


        if not delta in [0.5, 0.1, 0.01, 0.001]:
            print("You must provide an acceptable value for delta.")
            sys.exit(-1)

        if delta == 0.5:
            conds = [
                M_ratio < 2.9e-5,
                (M_ratio >= 2.9e-5) & (M_ratio < 3.3e-3),
                (M_ratio >= 3.3e-3) & (M_ratio < 5.3e-3),
                M_ratio >= 5.3e-3,
            ]
            func_epsilon = [
                lambda M_ratio: 1.58 * (100 * M_ratio) ** 0.65,
                lambda M_ratio: 0.055 * (100 * M_ratio) ** 0.076,
                lambda M_ratio: 0.17 * (100 * M_ratio) ** 1.12,
                lambda M_ratio: 0.1,
            ]

        if delta == 0.1:
            conds = [
                M_ratio < 9.4e-5,
                (M_ratio >= 9.4e-5) & (M_ratio < 5.0e-3),
                (M_ratio >= 5.0e-3) & (M_ratio < 6.6e-3),
                M_ratio >= 6.6e-3,
            ]
            func_epsilon = [
                lambda M_ratio: 0.12 * (100 * M_ratio) ** 0.59,
                lambda M_ratio: 0.026 * (100 * M_ratio) ** 0.27,
                lambda M_ratio: 0.50 * (100 * M_ratio) ** 4.53,
                lambda M_ratio: 0.1,
            ]

        if delta == 1.0e-2:
            conds = [
                M_ratio < 1.6e-5,
                (M_ratio >= 1.6e-5) & (M_ratio < 5.3e-3),
                (M_ratio >= 5.3e-3) & (M_ratio < 7.1e-3),
                M_ratio >= 7.1e-3,
            ]
            func_epsilon = [
                lambda M_ratio: 0.069 * (100 * M_ratio) ** 0.69,
                lambda M_ratio: 0.027 * (100 * M_ratio) ** 0.54,
                lambda M_ratio: 0.42 * (100 * M_ratio) ** 4.85,
                lambda M_ratio: 0.1,
            ]

        if delta == 1.0e-3:
            conds = [
                M_ratio < 7.6e-5,
                (M_ratio >= 7.6e-5) & (M_ratio < 4.5e-3),
                (M_ratio >= 4.5e-3) & (M_ratio < 7.1e-3),
                M_ratio >= 7.1e-3,
            ]
            func_epsilon = [
                lambda M_ratio: 0.065 * (100 * M_ratio) ** 0.71,
                lambda M_ratio: 0.020 * (100 * M_ratio) ** 0.47,
                lambda M_ratio: 0.26 * (100 * M_ratio) ** 3.67,
                lambda M_ratio: 0.1,
            ]
        return np.piecewise(M_ratio, conds, func_epsilon)

In [ ]:
#colors = ["#9400D3","#0000FF","#00FF00","#FFFF00",'#FF7F00','#FF0000']
colors = ["#461969ff","#3c4f8aff","#33638DFF","#29788eff","#25a186ff","#5cc863ff","#c3df2eff","#fde725ff"]
fig,(ax,bx) = plt.subplots(1,2,figsize=(20,10))

periods = [ 4/24, 6/24, 0.5, 1. ,2.,3.,5., 7.]

period_labels = ["4 Hours","6 Hours","12 Hours","1 Day","2 Days","3 Days","5 Days","1 Week"]

# Initiate 
Xray = xray(df,periods[0])

xray_L = np.log10(Xray.neutron_star_luminosity.value)

bx.plot(Xray.M_stripped / const.M_sun,xray_L,label="Neutron Star: 1.5 $v_{esc}$, 0.1 $dM_{wind}$",c="#28262C",ls="--")

ax.fill_between(np.arange(1.5,8),np.repeat(32,len(np.arange(1.5,8))),alpha=0.2,color="#1f0954",zorder=1)
#ax.fill_between(np.arange(1.5,8),np.repeat(32,len(np.arange(1.5,8))),alpha=0.2,color="",zorder=1)


ax.fill_between(np.arange(1.5,8),np.repeat(33.3,len(np.arange(1.5,8))),alpha=0.1,color="#fd798f",zorder=0)

# Source Luminosity
lmc2273 = np.log10(np.mean(np.array([4.15e33,2.2e33]))) #2273
smc2857 = np.log10(np.mean(np.array([2.46e34,1.24e34,1.07e34,6.08e33,8.16e33]))) #m2
smc4332 = np.log10(np.mean(np.array([1.12e34,2.53e34,0.57e34]))) #m1
smc6519 = np.log10(np.mean(np.array([7.64e34,7.27e34,4.55e34,2.75e34,2.06e34,3.62e34,7.10e34,2.64e34,0.39e34]))) #6519
source_L = [lmc2273,smc2857,smc4332,smc6519]
source_M = [3.1,2.5,2.5,2.2]
source_err = []

obsv_color = '#ff4d00'

ax.scatter(source_M,source_L,s=300,c=obsv_color,edgecolor="black",zorder=10)
ax.scatter(3.6,np.log10(1.53e33),s=300,c=obsv_color,edgecolor="black",zorder=10) #835


ax.plot([3.1,3.1],[np.log10(2.2e33),np.log10(4.15e33)],lw=5,c=obsv_color,zorder=9)
ax.plot([2.5,2.5],[np.log10(6.08e33),np.log10(2.46e34)],lw=5,c=obsv_color,zorder=9)
ax.plot([2.5,2.5],[np.log10(0.57e34),np.log10(2.53e34)],lw=5,c=obsv_color,zorder=9)
ax.plot([2.2,2.2],[np.log10(0.39e34),np.log10(7.64e34)],lw=5,c=obsv_color,zorder=9)

ax.scatter(5.5,np.log10(4.9e33),marker="v",s=300,c=obsv_color,edgecolor="black",zorder=10) #156
ax.scatter(2.3,np.log10(1.12e33),marker="v",s=300,c=obsv_color,edgecolor="black",zorder=10)  #m02
ax.scatter(2.7,np.log10(1.35e33),marker="v",s=300,c=obsv_color,edgecolor="black",zorder=10) #4349
ax.scatter(2.3,np.log10(9.49e32),marker="v",s=300,c=obsv_color,edgecolor="black",zorder=10) #206
ax.scatter(7.1,np.log10(6.71e33),marker="v",s=300,c=obsv_color,edgecolor="black",zorder=10) #a18
ax.scatter(2.5,np.log10(3.5e33),marker="v",s=300,c=obsv_color,edgecolor="black",zorder=10) #6198

for single_period,label,color in zip(periods,period_labels,colors):
    
    Xray = xray(df,single_period)

    Xray_L = np.log10(Xray.neutron_star_luminosity.value)
    
    ax.plot(Xray.M_stripped / const.M_sun, Xray_L,label=label,c=color,ls="--",lw=6)


for axes in [ax,bx]:
    axes.set_xlabel("$M_{\\mathrm{strip}}$ [$M_{\\odot}$]",fontsize=labelsize)
    axes.set_ylabel("$\\log_{10} L_X$ [erg/s]",fontsize=labelsize)
    


    lg = axes.legend(fontsize=ticksize,loc="lower right",title="Period",ncol=2)
    lg.get_title().set_fontsize(ticksize)
    axes.grid(c="gray", alpha=0.2)





    axes.set_xlim(1.6,7.2)
    axes.set_ylim(29,36)
    axes.set_xticks(ticks=[2,3,4,5,6,7])
    axes.set_xticklabels([2,3,4,5,6,7], fontsize=ticksize)
    axes.set_yticks([30,31,32,33,34,35,36])
    axes.set_yticklabels([30,31,32,33,34,35,36], fontsize=ticksize)

ax.text(7.25,32.15,"XMM Limit",fontsize=14,rotation=-90,c="black")
ax.text(7.25,29.55,"Chandra 50 ks limit",fontsize=14,rotation=-90,c="black")
ax.set_title(r"Neutron Star: 1.5 $v_{esc}$, 0.1 $dM_{wind}$",fontsize=labelsize)    

In [ ]:
np.log10(2e33)

In [ ]:
labelsize = 26
ticksize = 20
textsize = 20
legendsize = 18
ypad = 15
titlesize = 40
progenitor_sep = -0.25
progenitorsize = 15


targetcolor = "#7575cf"
sky2 = np.array([84, 107, 171]) / 255.0  # color of stripped stars
blue2 = np.array([9, 7, 52]) / 255.0  # Color of other stars
blue1 = np.array([13, 193, 231]) / 255.0  # Background for UV excess (apply alpha=0.03)
violet2 = np.array([161, 25, 107]) / 255.0  # models
royalblue = "#0000ff"
# xcand "#0099b5"
purple1 = np.array([54, 27, 77]) / 255
purple2 = np.array([164,162,197])/255

lightblue = np.array([88, 149, 169]) / 255
darkblue = np.array([51, 73, 110]) / 255
mediumblue = np.array([164, 162, 197]) / 255  # np.array([51,73,110])*2/255

targetcolor = "#7575cf"

def rgba(r,g,b,a):
    return(r/255,g/255,b/255,a)

orig = rgba(178,178,255,1)

colors = ['r','b','g','purple','navy']
colors=[sky2,blue2,blue1,violet2,royalblue,purple1,purple2,mediumblue]
colors = ["#222c37","#00cccc","#fff600","#ff0066","#19e3b1","#ff7f33","#b83c82"]
colors=['#F2671F','#C91B26','#9C0F5F','#60047A','#160A47',"#19e3b1","#222c37"]
colors=["#FF3855","#FA5B3D","#299617","#A7F432","#2243B6","#FF5470","#9C51B6"]
colors = ["#9400D3","#4B0082","#0000FF","#00FF00","#FFFF00",'#FF7F00','#FF0000']
colors = ["#440154ff","#461969ff","#461d6dff","#3cb875ff","#8dd544ff","#97d73eff","#fde725ff"]
colors = ["#461969ff","#3c4f8aff","#29788eff","#25a186ff","#5cc863ff","#c3df2eff","#fde725ff"]